In [31]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
import praw
from scipy.sparse import csr_matrix
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression

In [6]:
tokenizer = RegexpTokenizer(r'[a-zA-Z]+')
# create English stop words list
en_stop = set(stopwords.words('english'))
# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

In [12]:
# get posts from a subreddit
user_agent = ("Mental Health 1.0 by /u/kakakuo ")
r = praw.Reddit(user_agent=user_agent)
subreddit_name = 'NewYork'
# get all submissions of subreddit 
submissions = r.get_subreddit(subreddit_name).get_top_from_all(limit = 1000)
# go through all submissions
corpus_list = []
for submission in submissions:
    if submission.selftext:
        corpus_list.append(submission.selftext)

In [42]:
for (N,i) in enumerate(corpus_list):
    f = open(subreddit_name + '/corpus_'+str(N),'w')
    f.write(i.encode('utf-8').strip())
    f.close()

In [18]:
# list for tokenized documents in loop
texts = []
# loop through document list
for i in corpus_list:
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)
    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop and len(i)>2]
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    # add tokens to list
    texts.append(stemmed_tokens)
dictionary = corpora.Dictionary.load('BoW.dict')
corpus = [dictionary.doc2bow(text) for text in texts]

In [25]:
data = []
row = []
col = []
for (N,c) in enumerate(corpus):
    for i in c:
        data.append(i[1])
        row.append(N)
        col.append(i[0])
BoW_feature = csr_matrix((data,(row,col)),shape=(len(corpus),len(dictionary.keys()))).toarray()

In [27]:
ldamodel = gensim.models.ldamodel.LdaModel.load('lda20.model')

In [28]:
doc_lda = ldamodel[corpus]
data = []
row = []
col = []
for (N,doc) in enumerate(doc_lda):
    for i in doc:
        data.append(i[1])
        row.append(N)
        col.append(i[0])
LDA_feature = csr_matrix((data,(row,col))).toarray()

In [30]:
whole_feature = numpy.hstack((BoW_feature,LDA_feature))

In [40]:
model = joblib.load('logistic_regression.model')
p = model.predict(whole_feature)

In [44]:
f_result = open(subreddit_name + '/result','w')
for (N,i) in enumerate(p):
    if i == 1:
        print "***************corpus " + str(N) + "*****************"
        print corpus_list[N]
        f_result.write("***************corpus " + str(N) + "*****************\n")
        f_result.write(corpus_list[N])
        f_result.write('\n')

***************corpus 0*****************
It really feels like most posts to this sub are not by people who subscribe to it, but rather visitors confusing it with /r/nyc.
***************corpus 2*****************
So I recently watched AllTime10's video of their creepiest websites and their #1 was about Surveillance Sites. Now, I've seen these before but I usually watch the ones like EarthCam. But upon looking through these cams of one of these sites, I've discovered something disturbing: The security cameras of your kindergartens and daycares are being streamed **live** over the internet for anyone to see.

I recently found [this one](https://www.reddit.com/r/Michigan/comments/3g2z4q/anyone_here_live_in_climax_michigan/) and was able to find them, as well as two others which I've also contacted, but there are a few I can't find.

Can you help me find this one? Here are some screenshots of the feed (I've blacked out the faces of the children):

* [Screenshot #1](http://i.imgur.com/1iF65eu